# Homework - 01
                                                                                            CAP 6629 Reinforcement Learning
                                                                                                         Dr. Chang-Hwan Lee

                                                                                                        by - Rishabh Lingam

In [1]:
import operator
import numpy as np
from prettytable import PrettyTable

In [2]:
rows, cols = 7, 7
no_states = rows * cols
no_actions = 4
LEFT, UP, RIGHT, DOWN = 0, 1, 2, 3
START = [0, 48]
STOP = [25, 38]
Blocks = [2, 5, 7, 9, 17, 20, 22, 26, 31, 37, 39, 42]

In [3]:
# utility functions
def get_row_col_index_0(position, t_rows, t_cols):
    row_idx = position // t_cols
    col_idx = (position % t_cols)
    return row_idx, col_idx

def get_row_col_index(position):
    return get_row_col_index_0(position, rows, cols)

def get_position_0(row_idx, col_idx, t_rows, t_cols):
    if row_idx < 0 :
        row_idx = 0
    elif row_idx == t_rows:
        row_idx = t_rows - 1
    if col_idx < 0:
        col_idx = 0
    elif col_idx == t_cols:
        col_idx = t_cols - 1
    return (row_idx * t_cols) + (col_idx)

def get_position(row_idx, col_idx):
    return get_position_0(row_idx, col_idx, rows, cols)

def can_go_left(position):
    row_idx, col_idx = get_row_col_index(position)
    lef_pos = get_position(row_idx, col_idx - 1)
    return (col_idx != 0) and (lef_pos not in Blocks)

def can_go_right(position):
    row_idx, col_idx = get_row_col_index(position)
    right_pos = get_position(row_idx, col_idx + 1)
    return (col_idx != cols - 1) and (right_pos not in Blocks)

def can_go_up(position):
    row_idx, col_idx = get_row_col_index(position)
    up_pos = get_position(row_idx - 1, col_idx)
    return (row_idx != 0) and (up_pos not in Blocks)

def can_go_down(position):
    row_idx, col_idx = get_row_col_index(position)
    down_pos = get_position(row_idx + 1, col_idx)
    return (row_idx != rows - 1) and (down_pos not in Blocks)

def get_possible_action_state(position):
    row_idx, col_idx = get_row_col_index(position)
    actions, states = [], []
    
    if can_go_left(position):
        actions.append(LEFT)
        states.append(get_position(row_idx, col_idx-1))
    if can_go_up(position):
        actions.append(UP)
        states.append(get_position(row_idx-1, col_idx))
    if can_go_right(position):
        actions.append(RIGHT)
        states.append(get_position(row_idx, col_idx + 1))
    if can_go_down(position):
        actions.append(DOWN)
        states.append(get_position(row_idx+1, col_idx))
        
    return actions, states

def get_string_policy(policy):
    n = len(policy)
    string_policy = np.chararray(n, itemsize=10, unicode=True)
    for s in range(n):
        max_idx = np.argmax(policy[s])
        if max_idx == 0:
            string_policy[s] = 'LEFT'
        elif max_idx == 1:
            string_policy[s] = 'UP'
        elif max_idx == 2:
            string_policy[s] = 'RIGHT'
        elif max_idx == 3:
            string_policy[s] = 'DOWN'
    
    string_policy[START] = 'START'
    string_policy[STOP] = 'STOP'
    string_policy[Blocks] = 'BLOCK'
    return string_policy

def get_as_table(policy):
    pTable = PrettyTable()
    pTable.clear()
    pol_temp = policy.reshape((rows, cols))
    n = len(pol_temp)
    for i in range(n):
        pTable.add_row(pol_temp[i], divider=True)
    pTable.header = False
    return pTable

In [4]:
# Transition probability is |S| x |S'| x |A| array
# T[i][j][k]= prob. moving from state i to j when doing action k
# moving out of boundary or to block stays in current state

T_deterministic = np.zeros((no_states, no_states, no_actions))

for position in range(0, no_states):
    row_idx, col_idx = get_row_col_index(position)
        
    left_pos = get_position(row_idx, col_idx - 1)
    right_pos = get_position(row_idx, col_idx + 1)
    up_pos = get_position(row_idx - 1, col_idx)
    down_pos = get_position(row_idx + 1, col_idx)

    T_deterministic[position, left_pos, LEFT] = 1
    T_deterministic[position, up_pos, UP] = 1
    T_deterministic[position, right_pos, RIGHT] = 1
    T_deterministic[position, down_pos, DOWN] = 1

In [5]:
# 1-2) probabiistic transition    
#  complete T matrix for probabilistic transition

T_probabilistic = np.zeros((no_states, no_states, no_actions))
alpha = 0.05
for position in range(0, no_states):
    row_idx, col_idx = get_row_col_index(position)
    
    left_pos = get_position(row_idx, col_idx - 1)
    right_pos = get_position(row_idx, col_idx + 1)
    up_pos = get_position(row_idx - 1, col_idx)
    down_pos = get_position(row_idx + 1, col_idx)
    
    T_probabilistic[position, left_pos, LEFT] = 1 - (4 * alpha)
    a, s = get_possible_action_state(left_pos)
    for i in range(len(s)):
        T_probabilistic[s[i], left_pos, a[i]] = alpha
    
    T_probabilistic[position, up_pos, UP] = 1 - (4 * alpha)
    a, s = get_possible_action_state(up_pos)
    for i in range(len(s)):
        T_probabilistic[s[i], up_pos, a[i]] = alpha
    
    T_probabilistic[position, right_pos, RIGHT] = 1 - (4 * alpha)
    a, s = get_possible_action_state(right_pos)
    for i in range(len(s)):
        T_probabilistic[s[i], right_pos, a[i]] = alpha
    
    T_probabilistic[position, down_pos, DOWN] = 1 - (4 * alpha)
    a, s = get_possible_action_state(down_pos)
    for i in range(len(s)):
        T_probabilistic[s[i], down_pos, a[i]] = alpha  

In [6]:
# 1-3) Reward function: |S| x |A| array
# R[i][j]= reward from state i and action j
# each move generates -1 reward

R = np.full((no_states, no_actions), -1)

In [7]:
gamma = 0.9

In [8]:
#Policy: |S| x |A| array
#P[i][j]= prob of choosing action j in state i
# 2-1) initialize policy P with uniform policy

uniform_P = np.full((no_states, no_actions), 0.25)

In [9]:
# 2-2) implement prediction (policy evaluation)
# compute V values from a policy
# implement prediction(policy evaluation) algorithm in slide page 7.

def policy_eval(policy, max_iter, V, T):
    for i in range(max_iter):
        V_temp = np.zeros(no_states)
        for s in range(no_states):
            v_s = 0
            for action in range(no_actions):
                q_sa = 0
                for s_prime in range(no_states):
                    q_sa += T[s, s_prime, action] * V[s_prime]
                q_sa *= gamma
                q_sa += R[s, action]
                q_sa *= policy[s, action]
                v_s += q_sa
            V_temp[s] = np.round(v_s, 4)
            V_temp[STOP] = 0
        V = np.copy(V_temp)
    return V

In [10]:
# 2-2) implement prediction (policy evaluation)
# compute V values from a policy
# implement prediction(policy evaluation) algorithm in slide page 7.

def policy_eval_max(V, max_iter, T):
    for i in range(max_iter):
        V_temp = np.zeros(no_states)
        for s in range(no_states):
            v_s = []
            for action in range(no_actions):
                q_sa = 0
                for s_prime in range(no_states):
                    q_sa += T[s, s_prime, action] * V[s_prime]
                q_sa *= gamma
                q_sa += R[s, action]
                v_s.append(q_sa)
            V_temp[s] = np.round(np.max(v_s), 4)
            V_temp[STOP] = 0
        if np.sum(V == V_temp) == no_states:
            break    
        V = np.copy(V_temp)
    return V

In [11]:
# 2-3) implement policy improvement with V value using greedy method
# The formula for choosing the best action using V value is given in question.

def extract_policy(V):
    '''
    Procedure to extract a policy from a value function
    pi <-- argmax_a R^a + gamma T^a V

    Inputs:
    V -- Value function: array of |S| entries

    Output:
    policy -- Policy array P
    '''

    # initialize random(uniform) policy
    new_policy = np.zeros((no_states, no_actions))
    
    for s in range(no_states):
        actions, states = get_possible_action_state(s)
        max_idx = np.argmax(V[states])
        max_state = states[max_idx]
        max_act = actions[max_idx]
        new_policy[s, max_act] = 1

    return new_policy

In [12]:
# 2-4) implement policy iteration method
# implement policy iteration in slide page 13

def policy_iter(in_policy, max_iter, T):
    
    '''    Policy iteration procedure: alternate between 
    1) policy evaluation (solve V^pi = R^pi + gamma T^pi V^pi) and 
    2) policy improvement (pi <-- argmax_a R^a + gamma T^a V^pi).

    Inputs:
    in_policy -- Initial policy
    max_iter -- maximum # of iterations: scalar (use a large number)

    Outputs: 
    policy -- Policy P
    V -- Value function: array of |S| entries
    no_iter -- the actual # of iterations peformed by policy iteration: scalar
    '''

    # Initialization P and V using np.zeros
    P = in_policy
    no_iter = 0
    V = np.zeros(no_states)
    for i in range(max_iter):
        no_iter += 1
        V = policy_eval(P, 50, V, T)
        P_temp = extract_policy(V)
        if np.sum(P == P_temp) == (no_states * no_actions):
            break
        P = np.copy(P_temp)

    return P, V, no_iter

In [13]:
# 2-5) implement value iteration method
# implement value iteration in slide page 23
def value_iter(in_V, max_iter, T):
    '''
    Value iteration procedure
    V <-- max_a R^a + gamma T^a V

    Inputs:
    in_V -- Initial value function: array of |S| entries
    max_iter -- limit on the # of iterations: scalar (use large number)

    Outputs: 
    V -- Value function: array of |S| entries
    no_iter -- the actual # of iterations peformed by policy iteration: scalar
    '''
        
    # Initialization V using np.zeros
    V = in_V
    no_iter = 0
    
    for i in range(max_iter):
        no_iter += 1
        V_temp = policy_eval_max(V, 1, T)
        if np.sum(V == V_temp) == no_states:
            break
        V = np.copy(V_temp)
        
    return [V, no_iter]

## Deterministic Transition Probability Matrix

In [14]:
max_itr = 500

print("--- POLICY ITERATION ---")
p_itr_p, p_itr_v, p_itr_itr = policy_iter(uniform_P, max_itr, T_deterministic)
print("Total Iterations = ", p_itr_itr)
print("\nState Vales:")
print(get_as_table(p_itr_v))
print("\nPolicy:")
print(get_as_table(get_string_policy(p_itr_p)))

--- POLICY ITERATION ---
Total Iterations =  3

State Vales:
+---------+---------+---------+--------+-------+--------+---------+
|  -5.217 | -4.6856 | -4.0951 | -3.439 | -2.71 | -3.439 | -4.0951 |
+---------+---------+---------+--------+-------+--------+---------+
| -4.6856 | -4.0951 |  -3.439 | -2.71  |  -1.9 | -2.71  |  -3.439 |
+---------+---------+---------+--------+-------+--------+---------+
| -4.0951 |  -3.439 |  -2.71  |  -1.9  |  -1.0 |  -1.9  |  -2.71  |
+---------+---------+---------+--------+-------+--------+---------+
| -4.6856 |  -2.71  |   -1.9  |  -1.0  |  0.0  |  -1.0  |  -3.439 |
+---------+---------+---------+--------+-------+--------+---------+
| -4.0951 |  -3.439 |  -2.71  |  -1.0  |  -1.0 |  -1.9  |  -2.71  |
+---------+---------+---------+--------+-------+--------+---------+
| -4.0951 |  -3.439 |   -1.0  |  0.0   |  -1.0 | -2.71  |  -3.439 |
+---------+---------+---------+--------+-------+--------+---------+
|  -3.439 |  -2.71  |   -1.9  |  -1.0  |  -1.9 | -2.71 

In [15]:
max_itr = 500

print("--- VALUE ITERATION ---")
v_rand = np.zeros(no_states)
v_itr = value_iter(v_rand, max_itr, T_deterministic)
print("Total Iterations = ", v_itr[1])
print("\nState Vales:")
print(get_as_table(v_itr[0]))
print("\nPolicy:")
print(get_as_table(get_string_policy(extract_policy(v_itr[0]))))

--- VALUE ITERATION ---
Total Iterations =  8

State Vales:
+---------+---------+---------+--------+-------+--------+---------+
|  -5.217 | -4.6856 | -4.0951 | -3.439 | -2.71 | -3.439 | -4.0951 |
+---------+---------+---------+--------+-------+--------+---------+
| -4.6856 | -4.0951 |  -3.439 | -2.71  |  -1.9 | -2.71  |  -3.439 |
+---------+---------+---------+--------+-------+--------+---------+
| -4.0951 |  -3.439 |  -2.71  |  -1.9  |  -1.0 |  -1.9  |  -2.71  |
+---------+---------+---------+--------+-------+--------+---------+
|  -3.439 |  -2.71  |   -1.9  |  -1.0  |  0.0  |  -1.0  |   -1.9  |
+---------+---------+---------+--------+-------+--------+---------+
|  -3.439 |  -2.71  |   -1.9  |  -1.0  |  -1.0 |  -1.9  |  -2.71  |
+---------+---------+---------+--------+-------+--------+---------+
|  -2.71  |   -1.9  |   -1.0  |  0.0   |  -1.0 |  -1.9  |  -2.71  |
+---------+---------+---------+--------+-------+--------+---------+
|  -3.439 |  -2.71  |   -1.9  |  -1.0  |  -1.9 | -2.71  

## Probabilistic Transition Probability Matrix

In [16]:
max_itr = 500

print("--- POLICY ITERATION ---")
p_itr_p, p_itr_v, p_itr_itr = policy_iter(uniform_P, max_itr, T_probabilistic)
print("Total Iterations = ", p_itr_itr)
print("\nState Vales:")
print(get_as_table(p_itr_v))
print("\nPolicy:")
print(get_as_table(get_string_policy(p_itr_p)))

--- POLICY ITERATION ---
Total Iterations =  4

State Vales:
+---------+---------+---------+---------+---------+---------+---------+
| -3.4096 | -3.3467 | -3.0414 | -2.8353 | -2.3589 | -2.6984 | -3.0079 |
+---------+---------+---------+---------+---------+---------+---------+
| -3.2383 | -3.2593 | -2.7888 | -2.4844 | -1.8873 | -2.4844 | -2.7888 |
+---------+---------+---------+---------+---------+---------+---------+
| -3.1087 | -2.9287 | -2.4844 | -1.7811 | -1.0849 | -1.8873 | -2.3589 |
+---------+---------+---------+---------+---------+---------+---------+
| -3.3471 | -2.3589 | -1.8873 | -1.0849 |   0.0   |   -1.0  | -2.7892 |
+---------+---------+---------+---------+---------+---------+---------+
| -3.0655 | -2.8688 | -2.4039 |   -1.0  |  -1.045 | -1.8576 | -2.3375 |
+---------+---------+---------+---------+---------+---------+---------+
| -3.0786 |  -2.887 |   -1.0  |   0.0   |   -1.0  | -2.4473 | -2.8071 |
+---------+---------+---------+---------+---------+---------+---------+
|  

In [17]:
max_itr = 500

print("--- VALUE ITERATION ---")
v_rand = np.zeros(no_states)
v_itr = value_iter(v_rand, max_itr, T_probabilistic)
print("Total Iterations = ", v_itr[1])
print("\nState Vales:")
print(get_as_table(v_itr[0]))
print("\nPolicy:")
print(get_as_table(get_string_policy(extract_policy(v_itr[0]))))

--- VALUE ITERATION ---
Total Iterations =  14

State Vales:
+---------+---------+---------+---------+---------+---------+---------+
| -3.2978 | -3.2396 | -2.9669 | -2.7318 | -2.3589 | -2.6984 | -2.9428 |
+---------+---------+---------+---------+---------+---------+---------+
| -3.1914 | -3.1105 | -2.7318 | -2.4053 | -1.8873 | -2.4388 | -2.7441 |
+---------+---------+---------+---------+---------+---------+---------+
| -3.0436 | -2.8384 | -2.4101 | -1.7811 | -1.0849 | -1.8297 | -2.2384 |
+---------+---------+---------+---------+---------+---------+---------+
| -2.6984 | -2.3589 | -1.8873 | -1.0849 |   0.0   |   -1.0  |  -1.72  |
+---------+---------+---------+---------+---------+---------+---------+
|  -2.742 | -2.4195 | -1.8049 |   -1.0  |  -1.045 | -1.8021 | -2.2975 |
+---------+---------+---------+---------+---------+---------+---------+
| -2.3134 | -1.8241 |   -1.0  |   0.0   |   -1.0  | -1.8241 | -2.3134 |
+---------+---------+---------+---------+---------+---------+---------+
| -